In [1]:
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
start = time.time()

In [2]:
with open ('training/labels.tsv', 'r') as f:
    txt = f.read()

In [3]:
range_of_posts = re.findall('[0-9]{6}', txt)
# txt

In [4]:
post_filenames = ["training/posts/post-{}.xml".format(id) for id in range_of_posts]

In [5]:
# post_filenames[0:10]

In [6]:
def return_elements(filename):
    # get board name
    with open(filename, 'r') as f:
        txt = f.read()
    soup = BeautifulSoup(txt, 'lxml')
    message = soup.message
    board_id = message.board_id.text
    post_time = message.post_time.text
    subject = message.subject.text
    post_id = message.id.text
    author_id = message.author.get('href').split('/')[-1]
    root_id = message.root.get('href').split('/')[-1]
    body_text, emoji_ids = parse_body(message) # what to get out of body
    d = {}
#     d[author_id] = 
    return [author_id, board_id, post_time, subject, post_id, root_id, body_text, emoji_ids]

def parse_body(message):
    counter = 0
    for content in message.contents:
        if content.name == 'last_edit_time':
            break
        counter += 1
    body = message.contents[counter+1]
    body_text = re.findall("[a-zA-Z0-9]+(?![^<]*\>)", body.string)
    body_text = list(filter(lambda a: a != 'nbsp', body_text))
    emojis = re.findall('id="[a-z]+"', body.string)
    emoji_ids = [emoji.split('"')[1] for emoji in emojis]
    if len(emoji_ids) == 0:
        emoji_ids = None
    return body_text, emoji_ids
        

In [7]:
list_of_parsings = [return_elements(filename) for filename in post_filenames]
df = pd.DataFrame(list_of_parsings)
df.columns = ['author_id', 'board_id', 'post_time', 'subject', 'post_id', 'root_id', 'body_text', 'emoji_ids']

In [8]:
df.head()
end = time.time()
# df.to_csv('training_df.csv')
print('{} seconds have passsed'.format((end-start)))

4.163532733917236 seconds have passsed


## making sure the body is parsed correctly


In [9]:
with open("training/posts/post-135294.xml") as f:
    post = f.read()

In [10]:
soup = BeautifulSoup(post, 'lxml')
message = soup.message

In [11]:
# print(' '.join(parse_body(message)[0]))
# print(message.contents[14])